In [0]:
import pandas as pd

from sklearn.neighbors import KDTree



In [0]:
df = pd.read_csv("https://raw.githubusercontent.com/bw-spotify-song-suggester-4/data-science/master/data/merged_data.csv")


In [3]:
df.head()


,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0


In [0]:
# drop dups

df = df.drop_duplicates(subset = 'track_id').reset_index().drop(['index'], axis=1)

In [5]:
df.shape

(130989, 17)

In [0]:
# features to train on

df_features = df.drop(['artist_name', 'track_id', 'track_name', 'popularity'], axis=1)

df_info = df[['artist_name', 'track_id', 'track_name']]

In [0]:
# dictionary mapping on track id 

val_index_map = {value : index for index, value in zip(range(len(df_info)), df_info['track_id'])}

In [8]:
val_index_map

{'2RM4jf1Xa9zPgMGRDiht8O': 0,
 '1tHDG53xJNGsItRA3vfVgs': 1,
 '6Wosx2euFPMT14UXiWudMy': 2,
 '3J2Jpw61sO7l6Hc7qdYV91': 3,
 '2jbYvQCyPgX3CdmAzeVeuS': 4,
 '26Y1lX7ZOpw9Ql3gGAlqLK': 5,
 '5eIyK73BrxHLnly4F9PWqg': 6,
 '13Mf2ZBpfNkgWJowvM5hXh': 7,
 '7BQaRTHk44DkMhIVNcXy2D': 8,
 '049RxG2laEl9U1PGYeIqLV': 9,
 '118GQ70Sp6pMqn6w1oKuki': 10,
 '6S7cr72a7a8RVAXzDCRj6m': 11,
 '7h2qWpMJzIVtiP30E8VDW4': 12,
 '3KVQFxJ5CWOcbxdpPYdi4o': 13,
 '0JjNrI1xmsTfhaiU1R6OVc': 14,
 '3HjTcZt29JUHg5m60QhlMw': 15,
 '42LWRdkWxM9aWmDImWvH6C': 16,
 '32dMH9MvlTJaABrPHY52Yb': 17,
 '5RCPsfzmEpTXMCTNk7wEfQ': 18,
 '0y0mwXrdEzjUK5Nq8GDPnY': 19,
 '3RSMqu36JZnmMkrnNmnqyd': 20,
 '1o0fkWCltFHVeFIRHqvR5b': 21,
 '2iGShSeV6WcDbez5SLJ2bJ': 22,
 '2rNTo0tGUMW6rn0uHzV5er': 23,
 '5Egkx8edirN0pR2R58C2ME': 24,
 '67r3lnzstENsRYlZWq6DYP': 25,
 '4X8W9SSu9D5MujoxwIwqw6': 26,
 '4lncSzeN8WOH2iHEO593iZ': 27,
 '1L67mcddFQ65MfA3wO3MHV': 28,
 '21DU83QG4jB4mQKh76X32h': 29,
 '08nyEVO684j7pcTAhEY2zJ': 30,
 '4LMVmlX8WXPu8OyPwzkNpR': 31,
 '7JYCpIzpoidDOnnm

In [0]:
# train tree on data

tree = KDTree(df_features, leaf_size=10)


In [12]:
tree.query(pd.DataFrame(df_features.iloc[11]).T, k=11, return_distance=False)

array([[   11,    23,  2293,  2069, 37270, 70631,  6281, 75678, 75774,
        11122, 26890]])

In [0]:
# query based on id

def nearest_by_id(id, k=3):
  return tree.query(df_features.iloc[[val_index_map[id]]], k=k)[1]

In [14]:
val_index_map['4EW3qCWFvZnokLwhMB7WHT']

692

In [17]:
nearest_by_id('4EW3qCWFvZnokLwhMB7WHT')

array([[  692, 48854, 66511]])

In [18]:
df_features.iloc[692]

acousticness             0.921000
danceability             0.208000
duration_ms         144333.000000
energy                   0.023500
instrumentalness         0.000033
key                      7.000000
liveness                 0.311000
loudness               -16.365000
mode                     1.000000
speechiness              0.038400
tempo                  131.707000
time_signature           4.000000
valence                  0.226000
Name: 692, dtype: float64

In [20]:
df_features.iloc[48854]

acousticness             0.000338
danceability             0.511000
duration_ms         144328.000000
energy                   0.686000
instrumentalness         0.762000
key                     11.000000
liveness                 0.561000
loudness               -13.076000
mode                     0.000000
speechiness              0.033900
tempo                  123.620000
time_signature           4.000000
valence                  0.615000
Name: 48854, dtype: float64

In [21]:
df_features.iloc[66511]

acousticness             0.000132
danceability             0.509000
duration_ms         144328.000000
energy                   0.582000
instrumentalness         0.669000
key                      2.000000
liveness                 0.448000
loudness               -14.582000
mode                     1.000000
speechiness              0.035200
tempo                  123.572000
time_signature           4.000000
valence                  0.228000
Name: 66511, dtype: float64

In [0]:
# duration and tempo are standout


In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
spotify_scaler = StandardScaler().fit(df_features)

# track ids as index

df_ss = pd.DataFrame(spotify_scaler.transform(df_features),
                     columns=df_features.columns,
                     index=df_info['track_id'].values)


In [24]:
df_features

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.1180
1,0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.3710
2,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.3820
3,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.6410
4,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.9280
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130984,0.820000,0.418,235771,0.204,0.936,8,0.0866,-13.522,1,0.0370,110.022,4,0.0375
130985,0.070400,0.614,209032,0.952,0.000,6,0.3620,-2.782,1,0.0718,148.079,4,0.8200
130986,0.064000,0.343,324920,0.487,0.000,2,0.1070,-9.462,1,0.0473,171.217,4,0.3650
130987,0.029200,0.380,423377,0.399,0.000,2,0.8910,-8.466,1,0.0321,125.800,3,0.1050


In [0]:
# tree

tree = KDTree(df_ss, leaf_size=10)

In [27]:
df_ss.loc[['4EW3qCWFvZnokLwhMB7WHT']]

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
4EW3qCWFvZnokLwhMB7WHT,1.673788,-1.965431,-0.555174,-2.096053,-0.621279,0.490439,0.691846,-0.97668,0.803011,-0.591864,0.405904,0.235377,-0.824792


In [0]:
def closest_by_id(id, k=3):
  return tree.query(df_ss.loc[[id]], k=k)[1]

In [29]:
closest_by_id('4EW3qCWFvZnokLwhMB7WHT')

array([[   692, 108661, 113478]])

In [30]:
df_features.iloc[692]

acousticness             0.921000
danceability             0.208000
duration_ms         144333.000000
energy                   0.023500
instrumentalness         0.000033
key                      7.000000
liveness                 0.311000
loudness               -16.365000
mode                     1.000000
speechiness              0.038400
tempo                  131.707000
time_signature           4.000000
valence                  0.226000
Name: 692, dtype: float64

In [31]:
df_features.iloc[108661]

acousticness            0.9830
danceability            0.2590
duration_ms         46747.0000
energy                  0.1310
instrumentalness        0.1560
key                     6.0000
liveness                0.3370
loudness              -16.2380
mode                    1.0000
speechiness             0.0342
tempo                 135.8770
time_signature          4.0000
valence                 0.2940
Name: 108661, dtype: float64

In [32]:
df_features.iloc[113478]

acousticness             0.92700
danceability             0.32600
duration_ms         154227.00000
energy                   0.19300
instrumentalness         0.00543
key                      6.00000
liveness                 0.31700
loudness               -19.89900
mode                     1.00000
speechiness              0.03350
tempo                  152.83200
time_signature           4.00000
valence                  0.25700
Name: 113478, dtype: float64

In [0]:
# these are a lot closer !

In [0]:
# pickle
# ...